# INSERTION DONNEES FINALES

###### 

In [1]:
# Import des librairies

import os
import pandas as pd
import psycopg2
import csv
import reprlib
import numpy
import json

######

## Préparation des données

In [2]:
csv_file = 'data/wikirum_final.csv'
wikirum_final = pd.read_csv(csv_file)
csv_file1 = 'data/rum_data_final.csv'
rum_data_final = pd.read_csv(csv_file1)
csv_file2 = 'data/test_json_final.csv'
test_json_final = pd.read_csv(csv_file2)
excel_file = 'data/INVENTAIRE_FINAL.xlsx'
inventaire_final = pd.read_excel(excel_file)

global_final = pd.concat([wikirum_final, rum_data_final, test_json_final, inventaire_final],  ignore_index=True)
global_final

,nom,producteur,origine,volume,degre,type
0,Cuba 5 Years,1731 fine & rare,Pays-Bas,70.0,46.0,Vieux
1,Cuba Dominican Republic (Spanish Caribbean XO),1731 fine & rare,Pays-Bas,70.0,46.0,Vieux
2,Guatemala Panama Belize (Central America XO),1731 fine & rare,Pays-Bas,70.0,46.0,Vieux
3,Panama 6 Years,1731 fine & rare,Pays-Bas,70.0,46.0,Vieux
4,Trinidad Barbados Jamaica (British West Indies...,1731 fine & rare,Pays-Bas,70.0,46.0,Vieux
...,...,...,...,...,...,...
8462,"2005, 12 ans, octobre 2017",Foursquare,Barbados,70.0,59.0,Vieux
8463,DOMINUS 10 years,Foursquare,Barbados,70.0,56.0,Vieux
8464,BOLOGNE BLACK CANE,Bologne,Guadeloupe,70.0,50.0,Blanc
8465,BARBANCOURT VSOP 4 ans,Barbancourt,Haiti,70.0,40.0,Vieux


In [3]:
global_final.to_csv('data/global_final.csv', index = False, header=True)

###### 

---------------

## Insertion des données

######

### Insertion origines

###### 

In [4]:
# Prod
# liste des origines pour insertions

import csv
# je crée une liste regroupant ma liste de pays
liste = []
with open('data/global_final.csv', newline='', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        # gestion des doublons
        if row[2] not in liste:
            liste.append(row[2])
liste = liste[1:]
# print(liste)
# print(len(liste))

In [5]:
# connexion à la bdd collection
conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection5"
)
cur = conn.cursor()

for pays in liste:
    cur.execute("SELECT origine.nom FROM origine WHERE origine.nom = (%s) ", (pays,))
    verif = cur.fetchone()
    if not verif:
        cur.execute("INSERT INTO origine (nom) VALUES(%s)", (pays,))

conn.commit()
print('insertion terminée')
conn.close()

insertion terminée


######

----------------

### Insertion producteurs

######

In [6]:
# je crée une liste regroupant les pays et les producteurs
liste_co = []
liste_producteur = []
liste_final = []
with open('data/global_final.csv', newline='', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        if row[1:3] not in liste_co:
            liste_co.append((row[1:3]))
liste_co = liste_co[1:]

In [7]:
# prod
# fonction d'insertion dans la table producteur
conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection5"
)
cur = conn.cursor()
for producteur, pays in liste_co:
#     dictionnaire.items()
#     print(producteur)
#     print(pays)
    cur.execute(f"""SELECT origine.code_origine 
        FROM origine
        WHERE origine.nom = '{pays}';
        """)
    
    origine = cur.fetchone()[0]
#     print(origine)
    #pays = str(pro[0])
#     print(pays)

    cur.execute(f'''INSERT INTO producteur (nom_producteur, code_origine) VALUES (%s, %s)''', (producteur, origine))
    
cur.close()
conn.commit()
print('insertion terminée')
conn.close()

insertion terminée


###### 

___________

### Insertion produits

###### 

In [8]:
# insertion fichier excel
csv_file = 'data/global_final.csv'
inventaire = pd.read_csv(csv_file)
inventaire.head()

,nom,producteur,origine,volume,degre,type
0,Cuba 5 Years,1731 fine & rare,Pays-Bas,70.0,46.0,Vieux
1,Cuba Dominican Republic (Spanish Caribbean XO),1731 fine & rare,Pays-Bas,70.0,46.0,Vieux
2,Guatemala Panama Belize (Central America XO),1731 fine & rare,Pays-Bas,70.0,46.0,Vieux
3,Panama 6 Years,1731 fine & rare,Pays-Bas,70.0,46.0,Vieux
4,Trinidad Barbados Jamaica (British West Indies...,1731 fine & rare,Pays-Bas,70.0,46.0,Vieux


In [9]:
# cchangement des colonnes
inventaire_produit =  inventaire[['nom', 'volume', 'degre', 'producteur', 'type']]

In [10]:
liste_inventaire = [list(x) for x in inventaire_produit.to_numpy()]
print(liste_inventaire[0:10])

[['Cuba 5 Years', 70.0, 46.0, '1731 fine & rare', 'Vieux'], ['Cuba Dominican Republic (Spanish Caribbean XO)', 70.0, 46.0, '1731 fine & rare', 'Vieux'], ['Guatemala Panama Belize (Central America XO)', 70.0, 46.0, '1731 fine & rare', 'Vieux'], ['Panama 6 Years', 70.0, 46.0, '1731 fine & rare', 'Vieux'], ['Trinidad Barbados Jamaica (British West Indies XO)', 70.0, 46.0, '1731 fine & rare', 'Vieux'], ['Caroni 12 Ans', 70.0, 46.0, 'Blackadder', 'Vieux'], ['Caroni 18 Ans', 70.0, 63.1, 'Blackadder', 'Vieux'], ['Fiji 2001-2012 - 11 Years', 70.0, 63.9, 'Blackadder', 'Vieux'], ['Guadeloupe Belvedere 1998-2015- 17 Years', 70.0, 57.5, 'Blackadder', 'Vieux'], ['Guadeloupe Belvedere 1998-2017- 19 Years', 70.0, 56.6, 'Blackadder', 'Vieux']]


In [11]:
nom = liste_inventaire[0][0]
volume = liste_inventaire[0][1]
degre = liste_inventaire[0][2]
producteur = liste_inventaire[0][3]
type_r = liste_inventaire[0][4]


print(nom)
print((volume))
print(degre)
print(producteur)
print(type_r)


Cuba 5 Years
70.0
46.0
1731 fine & rare
Vieux


In [12]:
conn = psycopg2.connect(
        user = "postgres",
        password = os.environ.get("pg_psw"),
        host = "localhost",
        port = "5432",
        database = "collection5"
)
cur = conn.cursor()

for nom, volume, degre, producteur, type_r in liste_inventaire:
    
#     print(nom)
#     print(volume)
#     print(degre)
#     print(producteur)
#     print(type_r)

    cur.execute("""SELECT producteur.code_producteur 
        FROM producteur
        WHERE producteur.nom_producteur = %s;
        """, (producteur, ))
    
    code = cur.fetchone()[0]

    if volume > 0 and degre > 0:
        cur.execute("INSERT INTO produit (nom_produit, volume, degre, type_rhum, code_producteur) VALUES (%s, %s, %s, %s, %s)", (nom, volume, degre, type_r, code))
    elif volume < 0:
        cur.execute(f"INSERT INTO produit (nom_produit, degre, type_rhum, code_producteur) VALUES (%s, %s, %s, %s)", (nom, degre, type_r, code))
    elif degre < 0:
        cur.execute(f"INSERT INTO produit (nom_produit, volume, type_rhum, code_producteur) VALUES (%s, %s, %s, %s)", (nom, volume, type_r, code))

    else:
         cur.execute(f"INSERT INTO produit (nom_produit, type_rhum, code_producteur) VALUES (%s, %s, %s)", (nom, type_r, code))
#         print(type(cur))   
cur.close()
conn.commit()
print('insertion produit terminée')
conn.close()

insertion produit terminée
